In [21]:
import pandas as pd
import numpy as np
import math

In [15]:
import nbimporter
import feature_generation
import pre_processing
import feature_selection

Importing Jupyter notebook from feature_selection.ipynb


# Deteccion y eliminacion de Outliers usando DBSCAN

In [35]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

In [22]:
train,test = pre_processing.load_featured_datasets()

In [23]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [24]:
train_selected = feature_selection.get_selected_dataframe(train)

In [27]:
X = train_selected.drop('precio', axis=1).values
Y = train_selected['precio'].values

In [34]:
X_scaled = RobustScaler().fit_transform(X) #Estandarizado de los datos para clusterizar

In [36]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [37]:
X_scaled_filled = imp.fit_transform(X_scaled)

In [43]:
# Clusterizador
outlier_detection = DBSCAN(eps = .2,metric="manhattan", min_samples = 5,n_jobs = 4)

In [44]:
clusters = outlier_detection.fit_predict(X_scaled_filled)

In [48]:
np.count_nonzero(clusters == -1)

239592

# Deteccion y eliminacion de outliers usando IsolationForest 

In [49]:
from sklearn.ensemble import IsolationForest

In [52]:
train,test = pre_processing.load_featured_datasets()
train['precio'] = train['precio'].map(lambda x: math.log(x))
train_selected = feature_selection.get_selected_dataframe(train)
X = train_selected.drop('precio', axis=1).values
Y = train_selected['precio'].values

In [65]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_filled = imp.fit_transform(X)

In [66]:
clf = IsolationForest(behaviour='new', max_samples=100,random_state=1, contamination='auto')

In [67]:
y_pred = clf.fit_predict(X_filled)

In [69]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [76]:
def purge_outliers(array):
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    X_filled = imp.fit_transform(array)
    clf = IsolationForest(behaviour='new', max_samples=100,random_state=1, contamination='auto')
    y_pred = clf.fit_predict(X_filled)
    seleccion = y_pred == 1
    return seleccion